In [1]:
from models.db import get_db, SessionLocal, engine
from models.schemas import Item, Supplier, SupplierItem, Transactions
from sqlalchemy import func, desc, case, or_, UUID, String, DateTime
from datetime import datetime
import uuid

In [2]:
db = SessionLocal()

In [3]:
item = db.query(Item.item_code, Item.item_name, Item.item_quantity, Item.updated_at).all()
item

[(UUID('45afdece-3f10-55b3-af79-682bb642e04d'), 'test', 0, datetime.datetime(2025, 7, 21, 18, 2, 50, 398373))]

In [4]:
db.query(Transactions.item_code, Transactions.quantity, Transactions.updated_at).where(Transactions.updated_at >= datetime(2025, 6, 22, 7, 0, 0)).all()

[]

In [41]:
last_updated_at = func.max(func.coalesce(Transactions.updated_at, Item.updated_at)).label("updated_at")
query = (
    db.query(
        Item.item_code,
        Item.item_name,
        (Item.item_quantity + func.sum(func.coalesce(Transactions.quantity, 0))).label("current_stock"),
        last_updated_at
    )
    .outerjoin(Transactions, Item.item_code == Transactions.item_code)
    .where(
        or_(
            Transactions.updated_at == None,
            Transactions.updated_at >= Item.updated_at
        )
    )
    .group_by(Item.item_code, Item.item_name)
    .order_by(desc(last_updated_at))
)
inventory = query.all()
inventory

[]

In [43]:
from sqlalchemy import case, func, or_, desc

quantity_case = func.sum(
    case(
        (Transactions.updated_at >= Item.updated_at, Transactions.quantity),
        else_=0
    )
)

last_updated_at = func.max(
    case(
        (Transactions.updated_at >= Item.updated_at, Transactions.updated_at),
        else_=Item.updated_at
    )
).label("updated_at")

query = (
    db.query(
        Item.item_code,
        Item.item_name,
        (Item.item_quantity + func.coalesce(quantity_case, 0)).label("current_stock"),
        last_updated_at
    )
    .outerjoin(Transactions, Item.item_code == Transactions.item_code)
    .group_by(Item.item_code, Item.item_name, Item.item_quantity, Item.updated_at)
    .order_by(desc(last_updated_at))
)

inventory = query.all()
inventory

[('b371bc46', 'u', 50, datetime.datetime(2025, 6, 22, 19, 21, 53, 966172)),
 ('0a6f4b8e', 'tf', 10, datetime.datetime(2025, 6, 22, 19, 11, 14, 398482))]

In [6]:
import pandas as pd
import pandas_gbq as gbq

In [7]:
df = pd.read_csv("data/settlement.csv")
df

,項目,日付,支払い先,金額,支払元口座,支払先銀行,支払予定\n年月,ｽﾃｰﾀｽ,手数料,特記事項,口座登録名
0,NaN,2025/06/29,クラレトレーディング,"31,350",八二,三菱UFJ,2025-06,手続済み,550.0,NaN,NaN
1,NaN,2025/06/29,マルモ機材,"15,831",八二,八二,2025-07,7月末払,NaN,口座未登録,NaN
2,NaN,2025/06/29,千代田発條,"68,962",八二,八二,2025-06,手続済み,550.0,NaN,NaN
3,NaN,2025/06/29,㈱UK,"121,121",八二,八二,2025-06,手続済み,330.0,口座登録,カ．ユ－ケ－マツモト
4,NaN,2025/06/29,創悠游,"259,105",八二,八二,2025-06,手続済み,330.0,口座登録,ユ．ソユウ
5,NaN,2025/06/29,エルマックス,"105,688",八二,長野県信用組合,2025-06,手続済み,550.0,口座登録,カ）エルマツクス
6,NaN,2025/06/29,桜田電機,"84,920",八二,八二,2025-06,手続済み,330.0,NaN,NaN
7,NaN,2025/06/29,松本旭興社,"33,045",八二,八二,2025-06,手続済み,330.0,NaN,NaN
8,NaN,2025/06/29,千代田発條,"68,962",八二,八二,2025-06,誤り無効,NaN,NaN,NaN
9,NaN,2025/06/29,ヒーテック,"7,876",八二,八二,2025-06,手続済み,110.0,NaN,NaN


In [8]:
domain = uuid.uuid5(uuid.NAMESPACE_DNS, "https://archive-square.com")
team_id = uuid.uuid5(domain, "日昇あずみ（株）")
team_id

UUID('07917cf2-19c5-58e5-a311-a6f7a18f2721')

In [9]:
df_supplier = df[["支払い先","支払元口座","支払先銀行","特記事項","口座登録名"]].drop_duplicates(subset=["支払い先"])#.to_csv("data/settlement_unique.csv", index=False)
df_supplier = df_supplier.dropna(subset=["支払い先"])
df_supplier = df_supplier.rename(columns={"支払い先":"supplier_name", "支払元口座":"bank_from", "支払先銀行":"bank_to", "特記事項":"note", "口座登録名":"account_name"})
df_supplier["supplier_code"] = df_supplier["supplier_name"].apply(lambda x: uuid.uuid5(team_id, str(x)))
df_supplier["team_id"] = team_id
df_supplier["created_at"] = datetime.now()
df_supplier["updated_at"] = datetime.now()
df_supplier = df_supplier[[
    "team_id",
    "supplier_code",
    "supplier_name",
    "bank_from",
    "bank_to",
    "note",
    "account_name",
    "created_at",
    "updated_at"
]]

# df_supplier.to_sql("supplier_master", con=engine, if_exists="replace", index=False)
df_supplier

,team_id,supplier_code,supplier_name,bank_from,bank_to,note,account_name,created_at,updated_at
0,07917cf2-19c5-58e5-a311-a6f7a18f2721,748595b2-70ff-506b-a783-9515bb0a0d23,クラレトレーディング,八二,三菱UFJ,NaN,NaN,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
1,07917cf2-19c5-58e5-a311-a6f7a18f2721,261dfce8-754b-5bf5-b177-7a7ca9a6498b,マルモ機材,八二,八二,口座未登録,NaN,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
2,07917cf2-19c5-58e5-a311-a6f7a18f2721,497f29ae-b262-576a-9522-b842c2875cf1,千代田発條,八二,八二,NaN,NaN,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
3,07917cf2-19c5-58e5-a311-a6f7a18f2721,074df717-302f-58d9-a9f7-2e83a5e0dad2,㈱UK,八二,八二,口座登録,カ．ユ－ケ－マツモト,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
4,07917cf2-19c5-58e5-a311-a6f7a18f2721,0d5489e3-9607-5794-b27c-6796a03f8b26,創悠游,八二,八二,口座登録,ユ．ソユウ,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
5,07917cf2-19c5-58e5-a311-a6f7a18f2721,dd37ec82-6e0a-538c-a2ea-8ac9c4b6c956,エルマックス,八二,長野県信用組合,口座登録,カ）エルマツクス,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
6,07917cf2-19c5-58e5-a311-a6f7a18f2721,c3204493-b6da-525e-a5cc-c0d0903888de,桜田電機,八二,八二,NaN,NaN,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
7,07917cf2-19c5-58e5-a311-a6f7a18f2721,325d024a-be58-58cf-ae3b-02f64e9d9092,松本旭興社,八二,八二,NaN,NaN,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
9,07917cf2-19c5-58e5-a311-a6f7a18f2721,6cee6d6a-c183-5c3c-a75c-1abc163e1396,ヒーテック,八二,八二,NaN,NaN,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982
10,07917cf2-19c5-58e5-a311-a6f7a18f2721,c4334e33-dea7-5de1-b253-88e982db28bf,信興鍍金工業所,八二,八二,口座登録,ユ．シンコウメツキコウギヨウジヨ,2025-07-22 11:47:10.827783,2025-07-22 11:47:10.827982


In [11]:
df_supplier.to_sql("supplier_master", con=engine, if_exists="append", index=False, dtype={
    "team_id": UUID,
    "supplier_code": UUID,
    "supplier_name": String,
    "bank_from": String,
    "bank_to": String,
    "note": String,
    "account_name": String,
    "created_at": DateTime,
    "updated_at": DateTime
})

21